In [25]:
import sys

sys.path.insert(0, "..")
from src.settings import StyleSettings
from src.data.data_tools import StyleDataset
import numpy as np
import seaborn as sns
from trax.shapes import signature #Toegevoegd door Mirjam Bleumink, notebook 2 vraag 4


In [26]:
settings = StyleSettings()
traindataset = StyleDataset([settings.trainpath])
testdataset = StyleDataset([settings.testpath])


In [27]:
from src.models import tokenizer

corpus = []
for i in range(len(traindataset)):
    x = tokenizer.clean(traindataset[i][0])
    corpus.append(x)
v = tokenizer.build_vocab(corpus, max=20000)
len(v)

2022-07-05 08:52:04.891 | INFO     | src.models.tokenizer:build_vocab:27 - Found 19306 tokens


19308

In [28]:
# TODO ~ about 4 lines of code
# 5 regels code toegevoegd Mirjam Bleumink, notebook 2 vraag 1
class StylePreprocessor(tokenizer.Preprocessor): 
    def cast_label(self, label: str) -> int:
        d = {"humor": 0, "reuters": 1, "wiki": 2, "proverbs": 3}
        label_int = d[label]
        return label_int
# Einde toevoeging Mirjam Bleumink


In [29]:
preprocessor = StylePreprocessor(max=15, vocab=v, clean=tokenizer.clean)



In [30]:
from src.data import data_tools

trainstreamer = data_tools.BaseDatastreamer(
    dataset=traindataset, batchsize=32, preprocessor=preprocessor
).stream()
teststreamer = data_tools.BaseDatastreamer(
    dataset=testdataset, batchsize=32, preprocessor=preprocessor
).stream()


In [31]:
x, y = next(trainstreamer)
type(x), type(y)

(torch.Tensor, torch.Tensor)

In [32]:
from src.models import metrics
import torch
# TODO ~ 2 lines of code
# Mirjam Bleumink antwoord op notebook 2 vraag 3
# Aanvullende code voor deze vraag (tbv precision) opgenomen in metrics.py

metrics=[metrics.Accuracy(),  metrics.F1Score(), metrics.Precision()]
loss_fn = torch.nn.CrossEntropyLoss()


In [33]:
# Mirjam Bleumink antwoord op notebook 2, vraag 4 - bonus oplossing: model in Trax met gin configuratie
# Hiervoor zijn 2 aanvullende bestanden toegevoegd:
# - src.models.trax_basemodel 
# - basemodel.gin

import gin
import trax
from trax import layers as tl
from src.models import trax_basemodel

data_pipeline = trax.data.Serial(trax_basemodel.Cast()) #Tensors veranderen in numpy omdat Trax niet met Tensors werkt
trainpipe = data_pipeline(trainstreamer)                #trainstreamer om te gebruiken met Trax
testpipe = data_pipeline(teststreamer)                  #teststreamer om te gebruiken met Trax
x, y = next(trainpipe)
print(type(x),  type(y))

trax_model = trax_basemodel.Trax_Basemodel(vocab_size=len(v), d_feature=128, d_out=4)
trax_model.init_weights_and_state(signature(x))
trax_basemodel.summary(trax_model, x)



<class 'numpy.ndarray'> <class 'numpy.ndarray'>
layer                  input                dtype output               dtype 
(0) Embedding_19308_128 (32, 15)           ( int32 ) | (32, 15, 128)      (float32)
(1) GRU_128             (32, 15, 128)      (float32) | (32, 15, 128)      (float32)
(2) BatchNorm           (32, 15, 128)      (float32) | (32, 15, 128)      (float32)
(3) AvgLast             (32, 15, 128)      (float32) | (32, 15)           (float32)
(4) Dense_4             (32, 15)           (float32) | (32, 4)            (float32)


/home/mladmin/.cache/pypoetry/virtualenvs/exam-22-DDG3aTJy-py3.9/lib/python3.9/site-packages/jax/_src/numpy/lax_numpy.py:1939: UserWarning: Explicitly requested dtype <class 'jax.numpy.int64'> requested in zeros is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  lax_internal._check_user_dtype_supported(dtype, "zeros")


ShapeDtype{shape:(32, 4), dtype:float32}

In [34]:
# Mirjam Bleumink antwoord op notebook 2, vraag 4 - trainen van bonus oplossing: model in Trax

from trax.supervised import training
from trax.supervised.lr_schedules import warmup_and_rsqrt_decay
import gin
import trax
from trax import layers as tl
from src.models import trax_basemodel
from src.data import data_tools
gin.parse_config_file("basemodel.gin")

lr = warmup_and_rsqrt_decay(10, 0.01)
ll = tl.CategoryCrossEntropy()
log_dir_trax = "../models/trax"
log_dir_trax = data_tools.dir_add_timestamp(log_dir_trax)

train_task = training.TrainTask(
    labeled_data=trainpipe,
    loss_layer=tl.CategoryCrossEntropy(),
    optimizer=trax.optimizers.Adam(),
    lr_schedule=lr
)

eval_task = training.EvalTask(
    labeled_data=testpipe, 
    metrics=[tl.CategoryAccuracy(), tl.CategoryCrossEntropy()], 
    n_eval_batches=25
)

loop = training.Loop(
    trax_model,
    train_task,
    eval_tasks=[eval_task],
    output_dir=log_dir_trax
)

loop.run (200)


2022-07-05 08:52:05.549 | INFO     | src.data.data_tools:dir_add_timestamp:66 - Logging to ../models/trax/20220705-0852
/home/mladmin/.cache/pypoetry/virtualenvs/exam-22-DDG3aTJy-py3.9/lib/python3.9/site-packages/jax/_src/lib/xla_bridge.py:514: UserWarning: jax.host_count has been renamed to jax.process_count. This alias will eventually be removed; please update your code.
  warnings.warn(
/home/mladmin/.cache/pypoetry/virtualenvs/exam-22-DDG3aTJy-py3.9/lib/python3.9/site-packages/trax/layers/base.py:851: FutureWarning: GzipFile was opened for writing, but this will change in future Python releases.  Specify the mode argument for opening it for writing.
  with gzip.GzipFile(fileobj=f, compresslevel=compresslevel) as gzipf:



Step      1: Total number of trainable weights: 2570178
Step      1: Ran 1 train steps in 3.48 secs
Step      1: train CategoryCrossEntropy |  1.40070558


/home/mladmin/.cache/pypoetry/virtualenvs/exam-22-DDG3aTJy-py3.9/lib/python3.9/site-packages/trax/supervised/training.py:1249: FutureWarning: GzipFile was opened for writing, but this will change in future Python releases.  Specify the mode argument for opening it for writing.
  with gzip_lib.GzipFile(fileobj=f, compresslevel=2) as gzipf:


Step      1: eval      CategoryAccuracy |  0.36750000
Step      1: eval  CategoryCrossEntropy |  1.40485046

Step    100: Ran 99 train steps in 6.18 secs
Step    100: train CategoryCrossEntropy |  1.05743742
Step    100: eval      CategoryAccuracy |  0.74125000
Step    100: eval  CategoryCrossEntropy |  0.73729781

Step    200: Ran 100 train steps in 3.93 secs
Step    200: train CategoryCrossEntropy |  0.61748552
Step    200: eval      CategoryAccuracy |  0.79250000
Step    200: eval  CategoryCrossEntropy |  0.52489612


In [35]:
#Mirjam Bleumink antwoord op notebook 2 vraag 5 - evalueren van bonus oplossing: model in Trax

from sklearn.metrics import confusion_matrix


y_true = []
y_pred = []

for _ in range(30):
    x, y = next(testpipe)
    yhat = trax_model(x)
    yhat = yhat.argmax(axis=-1)
    y_pred.append(yhat.tolist())
    y_true.append(y.tolist())

yhat = [x for y in y_pred for x in y]
y = [x for y in y_true for x in y]

y, yhat

cfm = confusion_matrix(y, yhat)
cfm_norm = cfm / np.sum(cfm, axis=1, keepdims=True)
plot = sns.heatmap(cfm_norm, annot=cfm_norm, fmt=".3f")
plot.set(xlabel="Predicted", ylabel="Target")



[Text(0.5, 23.52222222222222, 'Predicted'),
 Text(50.722222222222214, 0.5, 'Target')]

# 6 Tune the model
Don't overdo this.
More is not better.

Bonus points for things like:
- Attention layers
- Trax architecture including a functioning training loop

Keep it small! It's better to present 2 or 3 sane experiments that are structured and thought trough, than 25 random guesses. You can test more, but select 2 or 3 of the best alternatives you researched, with a rationale why this works better.

Keep it concise; explain:
- what you changed
- why you thought that was a good idea  
- what the impact was (visualise or numeric)
- explain the impact

You dont need to get a perfect score; curiousity driven research that fails is fine.
The insight into what is happening is more important than the quantity.

Keep logs of your settings;
either use gin, or save configs, or both :)
Store images in the `figures` folder.

## <font color=green> Tuning van het model</font>
### Aanpak en componenten van het tunen
Uit de eerdere analyse van de dataset blijkt dat er een groot risico is op overfitting. Hiervoor heb ik enkele maatregelen vastgesteld, waarvan ik enkele in het model wil toepassen:
* _Vroeg stoppen_; hiervoor wil ik testen met diverse epoch lengtes zoals beschreven in notebook 2, vraag 4, Ik begin bij 50 en hoog op tot aan 1000, tenzij er eerder al een verlaging in de accuratesse plaats vindt.  
* _Reguralisatie door het toevoegen van een dropout in een attention laag_; hiervoor zal ik testen hoe het model presteert zonder attention laag, met attention laag zonder dropout en met attention laag met dropout.
* _Gebruik een simpel model_; alhoewel ik in de voorgaande test (dropout in een attention laag) aangeef dat ik een attention laag wil doorvoeren, wil ik ook testen hoe het model presteert zonder attenton laag. Als de accuratesse gelijk is met en zonder attention laag (al dan niet met dropout), dan zal ik kiezen voor het simpele model zonder attention omdat dit beter is voor het voorkomen/minimaliseren van overfitting. 
* _Toevoegen van synthetische voorbeelden (SMOTE)_; alhoewel dit waarschijnlijk een hele goede methode is om meer waarden voor de categorie proverbs te genereren en daarmee de dataset beter verdeeld te krijgen over de categorieën, heb ik besloten dat dit een activiteit is die ik niet zal doorvoeren in mijn tuning excersitie. Ik heb onderzoek gedaan naar SMOTE en imbalanced data in Trax, maar helaas is daar geen oplossing voor beschikbaar in Trax. Theoretisch zou het wel kunnen met gebruikmaking van SMOTE van imblearn.over_sampling, welke te gebruiken is met numpy arrays. ik heb echter besloten deze stap niet uit te voeren tbv de tijd. 

Ik wil graag enkele tuning activiteiten uitvoeren die specifiek mogelijk zijn doordat ik Trax gebruik, met name op het gebied van de learning rates. Trax biedt 4 methodes:
1. _trax.supervised.lr_schedules.constant_
    Deze learning rate vraagt om een _constant_, ik zal testen met de waarden 0.01, 0.001 en 0.0001
2. _trax.supervised.lr_schedules.warmup_
    Deze learning rate vraagt om een _max_value_, ik zal testen met de waarden 0.01, 0.001 en 0.0001. De waarde voor n_warmup_steps is 10. 
3. _trax.supervised.lr_schedules.warmup_and_rsqrt_decay_
    Deze learning rate vraagt om een _max_value_, ik zal testen met de waarden 0.01, 0.001 en 0.0001. De waarde voor n_warmup_steps is 10. 
4. _trax.supervised.lr_schedules.multifactor_
    Deze learning rate heeft heel veel configuratie mogelijkheden, welke ik niet allemaal zal uitproberen. Ik zal testen met 

Omdat de learning rate nauw verweven is met de gebruikte optimizer, ben ik voornemens om de learning rates te testen ten opzichte van de diverse optimizers die in trax worden geboden:
1. trax.optimizers.adafactor.Adafactor
2. trax.optimizers.adam.Adam
3. trax.optimizers.base.SGD
4. trax.optimizers.momentum.Momentum
5. trax.optimizers.rms_prop.RMSProp

Bij het tunen maak ik initieel gebruik van het Trax_basemodel, zoals deze eerder in het notebook is opgezet. Deze is als volgt opgebouwd:

<img src="../figures/trax_basemodel.png">

### Verwachtingen
#### Vroeg stoppen; epoch lengte
De verwachting is dat er een delicate balans zal zijn tussen lang genoeg trainen om een goed getraind model te hebben, maar niet te lang trainen waardoor het model gaat overfitten. Korter trainen zal mogelijk een negatief gevolg hebben voor de accuratesse van de categorieën 0, 1 en 2, maar van wezenlijk belang zijn voor het voorkomen van overfitten van de categorie 3 en daarmee het generaliserend vermogen van het model. 

#### Learning rate en optimizers
Ik moet eerlijk toegeven dat ik niet echt van tevoren kan bedenken welke learning rate methode en optimizer combinatie het beste zal werken, anders dan dat een variabele methode (methode 2, 3 of 4) waarschijnlijk beter zal presteren dan een constante. Dit experiment is meer ingegeven door interesse/nieuwsgierigheid in de mogelijkheden van Trax en de uitdaging die het mij biedt om dit in Trax uit te voeren. 

#### Attention laag (met en zonder dropout) & gebruik van een simpel model
Normaliter zou ik zeggen dat een attention laag toevoegen beter is voor het model. Ik denk dat dit voor de categorieën 0, 1 en 2 ook wel het geval zal zijn. Echter komt hier ook weer categorie 3 om de hoek kijken. 
Ik denk dat hier de evaluatie van een model zonder attention, met attention maar zonder dropout en met attention met dropout neerkomt op de gevolgen voor categorie 3. Indien het model even goed presteert met en zonder attention, zal ik kiezen voor een simpeler model zonder attention. 

### Resultaten
#### Vroeg stoppen; epoch lengte
De uitkomsten van de testen met de epoch lengte zijn in de volgende tabel inzichtelijk gemaakt. In de tabel staat de algemene locatie van alle logbestanden opgenomen (examen-22/examen-22/models/trax/epoch_lengte), evenals per test de folder waar de logbestanden van die test te vinden zijn. 

<img src="../figures/epoch_length.png">

In de experimenten met de epoch lengte heb ik bij een lengte 550 al geconstateerd dat de accuracy daalde, waardoor doortesten tot aan 1000 niet nodig was. 
In de eerste set van testen (met ophoging van 50) constateerde ik dat tussen de waarden 350 en 500 de hoogste accuracy werd behaald (*zie bovenste sectie van tabel). Ik heb vervolgens besloten in de reeks van 350 tot 500 met ophoging van 10 per keer te testen, om een optimale waarde voor de epoch lengte te bepalen. Uiteindelijk kwam ik op de beste accuracy uit bij een lengte van 450 (zie onderste sectie van de tabel). In de volgende test (learning rate en optimizer) maak ik daarom gebruik van een epoch lengte van 450. 

Hieronder volgt de grafische weergave van de train- en evaluatie activiteiten van de test lengte 450 vanuit tensorboard:

<img src="../figures/Lengte_450.png">

#### Learning rate en optimizers
De uitkomsten van de testen met de learning rates en optimizers zijn in de volgende tabel inzichtelijk gemaakt. In de tabel staat de algemene locatie van alle logbestanden opgenomen (examen-22/examen-22/models/trax/learningrate_optimizer), evenals per test de folder waar de logbestanden van die test te vinden zijn. In de tabel is tevens per learning rate methode een heatmap opgenomen om inzichtelijk te maken hoe de performance is met de verschillende configuratie opties ten opzichte van de optimizers. Na de tabel volgt de heatmap van alle learning rate methodes en configuraties ten opzichte van de optimizers. 

<img src="../figures/lr_optimizer.png">

<img src="../figures/lr_optimizer_heatmap.png">

Mijn verwachting was dat de multifactor uiteindelijk de beste zou zijn, aangezien die gebruik maakt van de andere 3 methodes en deze combineert tot een nieuwe learning rate, een beetje zoals "het beste van 3 werelden". Uit de resultaten blijkt echter dat de warmup and rsqrt decay met max_value 0.01 en Adam als optimizer het beste presteert (accuracy van 0.856 en loss van 0.430). Wat mij wel opvalt is dat ondanks de lagere accuracy van de multifactor methode met Adam als optimizer (0.845), de loss waarde wel de laagste is (0.416). Dat is wel logisch, want wijzigingen in de learning rate zijn juist bedoeld om een lagere loss te behalen.    

Hieronder volgt de grafische weergave van de train- en evaluatie activiteiten van de test met Adam optimizer, warmup and rsqrt decay methode en max_value van 0.01 vanuit tensorboard:

<img src="../figures/Adam_rsqrt-0_01.png">

Omdat de multifactor methode met Adam als optimizer een lagere loss had en hiermee een goede 2e plaats heeft behaald, laat ik hier ook de grafiesche weergave vanuit tensorboard van zien:

<img src="../figures/Adam_multifactor.png">

De logbestanden van de overige testen zijn allemaal te vinden in de folder welke per test is aangegeven in de resultaten tabel. 

#### Attention laag (met en zonder dropout) & gebruik van een simpel model 
Ik had hier heel graag willen melden dat ik een Trax model heb inclusief werkende attention laag, maar helaas heeft Trax mij hierin verslagen (of eigenlijk is mijn Trax kennis hierin tekort geschoten). 
Ik heb wel een attention laag toegevoegd in Trax_AttentionModel, welke te vinden is in trax_basemodel.py. 
Dit is de visuele weergave van dit model:

<img src="../figures/trax_attentionmodel.png">

Helaas krijg ik bij het trainen een foutmelding, die ik ondanks vele pogingen niet kan oplossen (zie ook de output van de cel genaamd "# Mirjam Bleumink cel gebruikt voor tuning activiteiten met attentionmodel"):

<img src="../figures/attention_model_error.png">

Ik heb diverse attention versies geprobeerd (PureAttention, AttentionQKV), inclusief en exclusief paralelle Dense lagen, voor en na de lagen van het basemodel. helaas heeft het allemaal niet geholpen.

De keuze is hiermee voor mij gemaakt; <font color=blue> het beste model op basis van de tuning activiteiten die ik heb uitgevoerd is het simpele model (zonder attention), met een epoch lengte van 450, Adam als optimizer en een warmup and rsqrt decay met max_value 0.01 als learning rate. </font>














### <font color=red> Hieronder volgen cellen die zijn gebruikt tijdens de tuning activiteiten. Deze zijn geen onderdeel van mijn antwoorden. </red>

In [36]:
# Mirjam Bleumink cel gebruikt voor tuning activiteiten met basemodel (zonder attention)

from trax.supervised import training
from trax.supervised.lr_schedules import warmup_and_rsqrt_decay
from trax.supervised.lr_schedules import constant
from trax.supervised.lr_schedules import warmup
from trax.supervised.lr_schedules import multifactor
import gin
import trax
from trax import layers as tl
from src.models import trax_basemodel
from src.data import data_tools
gin.parse_config_file("basemodel.gin")

#lr = warmup_and_rsqrt_decay(10, 0.0001)
#lr = constant(0.0001)
lr = warmup(10, 0.1)
#lr = multifactor (factors='constant * linear_warmup * rsqrt_decay', constant=0.1, warmup_steps=10, decay_factor=0.5, steps_per_decay=10, steps_per_cycle=100, second_constant=0.01, second_constant_step=10, minimum=0)

opt = trax.optimizers.Adam()
#opt = trax.optimizers.Adafactor()
#opt = trax.optimizers.Momentum() 
#opt = trax.optimizers.RMSProp()
#opt = trax.optimizers.SGD()

ll = tl.CategoryCrossEntropy()
log_dir_trax = "../models/trax"
log_dir_trax = data_tools.dir_add_timestamp(log_dir_trax)

train_task = training.TrainTask(
    labeled_data=trainpipe,
    loss_layer=tl.CategoryCrossEntropy(),
    optimizer=opt,
    lr_schedule=lr
)

eval_task = training.EvalTask(
    labeled_data=testpipe, 
    metrics=[tl.CategoryAccuracy(), tl.CategoryCrossEntropy()], 
    n_eval_batches=25
)

loop = training.Loop(
    trax_model,
    train_task,
    eval_tasks=[eval_task],
    output_dir=log_dir_trax
)

loop.run (450)


2022-07-05 08:52:26.165 | INFO     | src.data.data_tools:dir_add_timestamp:66 - Logging to ../models/trax/20220705-0852
/home/mladmin/.cache/pypoetry/virtualenvs/exam-22-DDG3aTJy-py3.9/lib/python3.9/site-packages/jax/_src/lib/xla_bridge.py:514: UserWarning: jax.host_count has been renamed to jax.process_count. This alias will eventually be removed; please update your code.
  warnings.warn(
/home/mladmin/.cache/pypoetry/virtualenvs/exam-22-DDG3aTJy-py3.9/lib/python3.9/site-packages/trax/supervised/training.py:1388: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  return [f for f in flat if f is not None and f is not ()]  # pylint: disable=literal-comparison
/home/mladmin/.cache/pypoetry/virtualenvs/exam-22-DDG3aTJy-py3.9/lib/python3.9/site-packages/trax/supervised/training.py:1388: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  return [f for f in flat if f is not None and f is not ()]  # pylint: disable=literal-comparison
/home/mladmin/.cache/pypoetry/virtualen

LayerError: Exception passing through layer Serial (in pure_fn):
  layer created in file [...]/trax/supervised/training.py, line 1033
  layer input shapes: (ShapeDtype{shape:(32, 15), dtype:int32}, ShapeDtype{shape:(32,), dtype:int32})

  File [...]/trax/layers/combinators.py, line 90, in forward
    outputs, s = layer.pure_fn(inputs, w, s, rng, use_cache=True)

LayerError: Exception passing through layer Serial (in pure_fn):
  layer created in file [...]/src/models/trax_basemodel.py, line 63
  layer input shapes: ShapeDtype{shape:(32, 15), dtype:int32}

  File [...]/trax/layers/combinators.py, line 90, in forward
    outputs, s = layer.pure_fn(inputs, w, s, rng, use_cache=True)

LayerError: Exception passing through layer BatchNorm (in pure_fn):
  layer created in file [...]/src/models/trax_basemodel.py, line 66
  layer input shapes: ShapeDtype{shape:(32, 15, 128), dtype:float32}

  File [...]/trax/layers/normalization.py, line 49, in forward
    n_batches += 1

  File [...]/site-pack

In [ ]:

# Mirjam Bleumink cel gebruikt voor tuning activiteiten met attentionmodel 

from trax.supervised import training
from trax.supervised.lr_schedules import warmup_and_rsqrt_decay
from trax.supervised.lr_schedules import constant
from trax.supervised.lr_schedules import warmup
from trax.supervised.lr_schedules import multifactor
import gin
import trax
from trax import layers as tl
from src.models import trax_basemodel
from src.data import data_tools

trax_attention_model = trax_basemodel.Trax_AttentionModel(vocab_size=len(v), d_feature=128, d_out=4, n_heads=1, dropout=0.0, mode='train')
print(trax_attention_model)
#trax_attention_model.init_weights_and_state(signature(x))
#trax_basemodel.summary(trax_model, x)

ll = tl.CategoryCrossEntropy()
log_dir_trax = "../models/trax"
log_dir_trax = data_tools.dir_add_timestamp(log_dir_trax)

train_task = training.TrainTask(
    labeled_data=trainpipe,
    loss_layer=tl.CategoryCrossEntropy(),
    optimizer=opt,
    lr_schedule=lr
)

eval_task = training.EvalTask(
    labeled_data=testpipe, 
    metrics=[tl.CategoryAccuracy(), tl.CategoryCrossEntropy()], 
    n_eval_batches=25
)

loop = training.Loop(
    trax_attention_model,
    train_task,
    eval_tasks=[eval_task],
    output_dir=log_dir_trax
)

loop.run (450)



2022-07-04 14:21:11.715 | INFO     | src.data.data_tools:dir_add_timestamp:67 - Logging to ../models/trax/20220704-1421


Serial_in4_out2[
  Embedding_19308_128
  GRU_128
  BatchNorm
  AvgLast
  Dense_4
  Parallel_in3_out3[
    Dense_128
    Dense_128
    Dense_128
  ]
  PureAttention_in4_out2
  Dense_128
  Serial[
    Relu
  ]
  Dense_4
]


LayerError: Exception passing through layer Serial (in init):
  layer created in file [...]/src/models/trax_basemodel.py, line 74
  layer input shapes: (ShapeDtype{shape:(32, 15), dtype:int32}, ShapeDtype{shape:(32,), dtype:int64})

  File [...]/trax/layers/combinators.py, line 107, in init_weights_and_state
    sublayer.init(inputs, use_cache=True))

LayerError: Exception passing through layer Parallel (in init):
  layer created in file [...]/src/models/trax_basemodel.py, line 80
  layer input shapes: (ShapeDtype{shape:(32, 4), dtype:float32}, ShapeDtype{shape:(32,), dtype:int64})

  File [...]/trax/layers/combinators.py, line 225, in init_weights_and_state
    sublayer_signatures = self._allot_to_sublayers(input_signature)

  File [...]/trax/layers/combinators.py, line 270, in _allot_to_sublayers
    sub_inputs.append(inputs[start])

IndexError: tuple index out of range

In [ ]:
#barplot epoch lengte
import seaborn as sn
import pandas as pd

array = [[0.268,
        0.743,
        0.676,
        0.775,
        0.830,
        0.844,
        0.848,
        0.825,
        0.874,
        0.853,
        0.839
        ]]
df_epochs = pd.DataFrame(array,['accuracy'],
                  columns = ['50','100','150','200','250','300','350','400','450','500','550'])

epochs = sn.barplot(data = df_epochs,palette = "Blues")
epochs.set(xlabel = 'epoch length', ylabel = 'accuracy')
epochs.bar_label(epochs.containers[0], size = 8)

# plt.figure(figsize = (10,7))
# sn.heatmap(df_cm, annot=True)

[Text(0, 0, '0.268'),
 Text(0, 0, '0.743'),
 Text(0, 0, '0.676'),
 Text(0, 0, '0.775'),
 Text(0, 0, '0.83'),
 Text(0, 0, '0.844'),
 Text(0, 0, '0.848'),
 Text(0, 0, '0.825'),
 Text(0, 0, '0.874'),
 Text(0, 0, '0.853'),
 Text(0, 0, '0.839')]

In [ ]:
#barplot epoch lengte - finetunen 350-500
import seaborn as sn
import pandas as pd

array = [[
        0.848,
        0.828,
        0.828,
        0.820,
        0.830,
        0.825,
        0.831,
        0.851,
        0.825,
        0.811,
        0.859,
        0.813,
        0.856,
        0.815,
        0.803,
        0.853
        ]]
df_epochs = pd.DataFrame(array,['accuracy'],
                  columns = ['350','360','370','380','390','400','410','420','430','440','450','460','470','480','490','500'])

epochs = sn.barplot(data = df_epochs,palette = "Blues")
epochs.set(xlabel = 'epoch length', ylabel = 'accuracy')
epochs.bar_label(epochs.containers[0], size = 8)

# plt.figure(figsize = (10,7))
# sn.heatmap(df_cm, annot=True)

[Text(0, 0, '0.268'),
 Text(0, 0, '0.743'),
 Text(0, 0, '0.676'),
 Text(0, 0, '0.775'),
 Text(0, 0, '0.83'),
 Text(0, 0, '0.844'),
 Text(0, 0, '0.848'),
 Text(0, 0, '0.825'),
 Text(0, 0, '0.874'),
 Text(0, 0, '0.853'),
 Text(0, 0, '0.839')]

In [ ]:
#barplot epoch lengte - finetunen 350-500
import seaborn as sn
import pandas as pd

array = [[
        0.848,
        0.828,
        0.828,
        0.820,
        0.830,
        0.825,
        0.831,
        0.851,
        0.825,
        0.811,
        0.859,
        0.813,
        0.856,
        0.815,
        0.803,
        0.853
        ]]
df_epochs = pd.DataFrame(array,['accuracy'],
                  columns = ['350','360','370','380','390','400','410','420','430','440','450','460','470','480','490','500'])

epochs = sn.barplot(data = df_epochs,palette = "Blues")
epochs.set(xlabel = 'epoch length', ylabel = 'accuracy')
epochs.bar_label(epochs.containers[0], size = 8)

# plt.figure(figsize = (10,7))
# sn.heatmap(df_cm, annot=True)

[Text(0, 0, '0.268'),
 Text(0, 0, '0.743'),
 Text(0, 0, '0.676'),
 Text(0, 0, '0.775'),
 Text(0, 0, '0.83'),
 Text(0, 0, '0.844'),
 Text(0, 0, '0.848'),
 Text(0, 0, '0.825'),
 Text(0, 0, '0.874'),
 Text(0, 0, '0.853'),
 Text(0, 0, '0.839')]

In [ ]:
#heatmap - learning rate & optimizer 
import seaborn as sn
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors



optimizers = ['Adam','Adafactor','Momentum','RMSProp','SGD']
learning_rates = ['constant 0.01','constant 0.001','constant 0.0001','warmup 0.01','warmup 0.001','warmup 0.0001','warmup and rsqrt_decay 0.01','warmup and rsqrt_decay 0.001','warmup and rsqrt_decay 0.0001','multifactor']

accuracy = np.array([
        [0.855,0.770,0.740,0.788,0.489],
        [0.796,0.549,0.608,0.756,0.438],
        [0.611,0.421,0.469,0.628,0.380],
        [0.825,0.764,0.749,0.838,0.674],
        [0.818,0.540,0.515,0.805,0.388],
        [0.511,0.389,0.463,0.643,0.441],
        [0.856,0.644,0.561,0.788,0.560],
        [0.690,0.423,0.513,0.603,0.452],
        [0.490,0.235,0.470,0.424,0.405],
        [0.845,0.781,0.734,0.820,0.581]     
        ])

cmap = colors.ListedColormap
#YlOrRd

fig, ax = plt.subplots()
heatmap = ax.imshow(accuracy, cmap = 'YlOrRd')

ax.set_xticks(np.arange(len(optimizers)), labels=optimizers)
ax.set_yticks(np.arange(len(learning_rates)), labels=learning_rates)

plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

for i in range(len(learning_rates)):
    for j in range(len(optimizers)):
        text = ax.text(j, i, accuracy[i, j],
                       ha="center", va="center", color="black", size = 6)         


plt.colorbar(heatmap)
             

In [ ]:
#heatmap - learning rate & optimizer 
import seaborn as sn
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors



optimizers = ['Adam','Adafactor','Momentum','RMSProp','SGD']
learning_rates = ['multifactor']

accuracy = np.array([
        #[0.855,0.770,0.740,0.788,0.489],
        #[0.796,0.549,0.608,0.756,0.438],
        #[0.611,0.421,0.469,0.628,0.380]#,
        # [0.825,0.764,0.749,0.838,0.674],
        # [0.818,0.540,0.515,0.805,0.388],
        # [0.511,0.389,0.463,0.643,0.441]#,
        #  [0.856,0.644,0.561,0.788,0.560],
        #  [0.690,0.423,0.513,0.603,0.452],
        #  [0.490,0.235,0.470,0.424,0.405]#,
         [0.845,0.781,0.734,0.820,0.581]     
        ])

cmap = colors.ListedColormap
#YlOrRd

fig, ax = plt.subplots()
heatmap = ax.imshow(accuracy, cmap = 'YlOrRd')

ax.set_xticks(np.arange(len(optimizers)), labels=optimizers)
ax.set_yticks(np.arange(len(learning_rates)), labels=learning_rates)

plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

for i in range(len(learning_rates)):
    for j in range(len(optimizers)):
        text = ax.text(j, i, accuracy[i, j],
                       ha="center", va="center", color="black", size = 6)         


plt.colorbar(heatmap)
             